# Scraping

this code provide how to scrap the [D&D 5e Wiki](https://dnd-5e.fandom.com/wiki/D%26D_5e_Wiki)

In [25]:
import requests
import re
import os
import string
import pandas as pd
import markdownify 

from io import StringIO
from bs4 import BeautifulSoup, Comment
from tqdm import tqdm
from IPython.display import display, clear_output

In [2]:
url = requests.get('https://dnd-5e.fandom.com/wiki/List_of_Spells')
soup = BeautifulSoup(url.content, 'html.parser')

# Spell

filter out `Unearthed Arcana`

In [3]:
data = soup.find_all('span', 'mw-headline')
filtered_data = [item for item in data if item.text != 'Unearthed Arcana' and item.text != 'Spell Navigation']

In [4]:
filtered_data

[<span class="mw-headline" id="Cantrips">Cantrips</span>,
 <span class="mw-headline" id="1st_Level">1st Level</span>,
 <span class="mw-headline" id="2nd_Level">2nd Level</span>,
 <span class="mw-headline" id="3rd_Level">3rd Level</span>,
 <span class="mw-headline" id="4th_Level">4th Level</span>,
 <span class="mw-headline" id="5th_Level">5th Level</span>,
 <span class="mw-headline" id="6th_Level">6th Level</span>,
 <span class="mw-headline" id="7th_Level">7th Level</span>,
 <span class="mw-headline" id="8th_Level">8th Level</span>,
 <span class="mw-headline" id="9th_Level">9th Level</span>]

In [5]:
dfs = []

for table in filtered_data:
    table = table.find_next('table')
    html_table = list()
    
    if table:
        html_string = str(table)
        html_table.append(table)
        df = pd.read_html(StringIO(html_string))[0]
        dfs.append(df)

print(len(dfs))
display(dfs[0].head())
display(dfs[1].head())

10


,Name,School,Casting Time,Comps,Save,Conc.,Source
0,Acid Splash,Conjuration,1 action,"V, S",Dexterity,NaN,PHB 211
1,Blade Ward,Abjuration,1 action,"V, S",NaN,NaN,PHB 218
2,Booming Blade,Evocation,1 action,"V, M",Attack roll,NaN,"TCE 143, SCAG 142"
3,Chill Touch,Necromancy,1 action,"V, S",Attack roll,NaN,PHB 221
4,Control Flames,Transmutation,1 action,S,NaN,NaN,"XGE 152, EEPC 16"


,Name,School,Casting Time,Comps,Cost,Save,Ritual,Conc.,Source
0,Absorb Elements,Abjuration,1 reaction,S,NaN,NaN,NaN,NaN,"XGE 150, EEPC 15"
1,Alarm,Abjuration,1 minute,"V, S, M",NaN,NaN,Yes,NaN,PHB 211
2,Animal Friendship,Enchantment,1 action,"V, S, M",NaN,Wisdom,NaN,NaN,PHB 212
3,Armor of Agathys,Abjuration,1 action,"V, S, M",NaN,NaN,NaN,NaN,PHB 215
4,Arms of Hadar,Conjuration,1 action,"V, S",NaN,Strength,NaN,NaN,PHB 215


In [6]:
os.makedirs("../datasets/spell/", exist_ok=True)

for title, df in zip(filtered_data, dfs):
    df.to_csv(f"../datasets/spell/{title.text}.csv", index=False)

In [7]:
filtered_data

[<span class="mw-headline" id="Cantrips">Cantrips</span>,
 <span class="mw-headline" id="1st_Level">1st Level</span>,
 <span class="mw-headline" id="2nd_Level">2nd Level</span>,
 <span class="mw-headline" id="3rd_Level">3rd Level</span>,
 <span class="mw-headline" id="4th_Level">4th Level</span>,
 <span class="mw-headline" id="5th_Level">5th Level</span>,
 <span class="mw-headline" id="6th_Level">6th Level</span>,
 <span class="mw-headline" id="7th_Level">7th Level</span>,
 <span class="mw-headline" id="8th_Level">8th Level</span>,
 <span class="mw-headline" id="9th_Level">9th Level</span>]

In [ ]:
data = soup.find_all('span', 'mw-headline')
filtered_data = [item for item in data if item.text != 'Unearthed Arcana' and item.text != 'Spell Navigation']  
os.makedirs("../datasets/spell_content", exist_ok=True)

for table in filtered_data:
    file_name = table.text
    table = table.find_next('table')
    data = list()

    if table:
        anchors = table.find_all('a')
        
        for anchor in tqdm(anchors):
            url = requests.get(f'https://dnd-5e.fandom.com{anchor['href']}')
            soup = BeautifulSoup(url.content, 'html.parser')
            name_tag = soup.find('span', class_='mw-page-title-main')

            h2_tags = soup.find_all('h2')

            for h2_tag in h2_tags:
                headline = h2_tag.find('span', class_='mw-headline')
                if headline:
                    new_h2 = soup.new_tag("h2")
                    new_h2.string = headline.text.strip()
                    h2_tag.replace_with(new_h2)

            content_block = soup.find('div', 'mw-content-ltr mw-parser-output')
            content_children = content_block.findChildren()
            cleaned_content = []

            for element in content_children:
                markdown = markdownify.markdownify(str(element), strip=['a', 'span'], heading_style="ATX")
                cleaned_content.append(markdown)

            combined_content = f"# {name_tag.text.strip()}\n## Spell Name\n{name_tag.text.strip()}\n\n" + "\n".join(cleaned_content) + "\n"
            combined_content = re.sub(r'<!--.*?-->', '', combined_content, flags=re.DOTALL)

            data.append(combined_content)

        with open(f"../datasets/spell_content/{file_name}.txt", "w", encoding="utf-8") as f:
            for item in data:
                f.write(f"{item}\n")

  0%|          | 0/46 [00:00<?, ?it/s]/tmp/ipykernel_20610/3706966594.py:28: DeprecationWarning: Call to deprecated method findChildren. (Replaced by find_all) -- Deprecated since version 3.0.0.
  content_children = content_block.findChildren()
100%|██████████| 22/22 [00:11<00:00,  1.95it/s]


In [ ]:
url = requests.get('https://dnd-5e.fandom.com/wiki/List_of_Spells')
soup = BeautifulSoup(url.content, 'html.parser')

In [32]:
def content(headers, category_url, base_url, output_dir):
    # Set to store unique URLs
    monster_links = dict()

    # Phase 1: Collect all unique monster links
    print("Collecting all unique monster links...")
    for letter in string.ascii_uppercase:
        print(f"Checking: {letter}")
        monster_set = set()
        url = category_url + letter
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch page for letter {letter}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.select('div.category-page__members a.category-page__member-link')

        for link in tqdm(links):
            href = link['href']
            full_url = f"{base_url}{href}"
            monster_set.add(full_url)
        
        monster_links[letter] = monster_set

    print(f"\nFinish")
    
    clear_output(wait=True)
    # Phase 2: Scrape each unique monster page and save as Markdown

    for letter, monster_set in sorted(monster_links.items()):
        markdown_text = ""

        for monster_url in tqdm(sorted(monster_set)):
            monster_resp = requests.get(monster_url, headers=headers)
            if monster_resp.status_code != 200:
                print(f"Failed to fetch {monster_url}")
                continue

            monster_soup = BeautifulSoup(monster_resp.text, 'html.parser')
            main_content = monster_soup.find('main', class_='page__main')
            if not main_content:
                print(f"Skipped (no main content): {monster_url}")
                continue

            # Remove unwanted elements
            for cls in ['page-side-tools__wrapper', 'page-header__actions', 'license-description', 'page-header__categories']:
                for tag in main_content.find_all('div', class_=cls):
                    tag.decompose()

            # Extract name from the page
            name_tag = monster_soup.select_one("span.mw-page-title-main")
            name = name_tag.text.strip() if name_tag else "Unknown Monster"

            # Convert to markdown
            markdown = markdownify.markdownify(str(main_content), strip=['a', 'span'], heading_style="ATX")
            markdown_text += f"{markdown}\n\n---\n\n"

        # Save grouped file for the letter
        filename = os.path.join(output_dir, f"{letter}.md")
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(markdown_text)

        print(f"Saved: {filename}")


In [33]:
base_url = "https://dnd-5e.fandom.com"
category_url = f"{base_url}/wiki/Category:Monsters?from="
headers = {"User-Agent": "Mozilla/5.0"}

output_dir = "../datasets/monsters_markdown"
os.makedirs(output_dir, exist_ok=True)

content(headers, category_url, base_url, output_dir)

Checking: A


100%|██████████| 98/98 [00:00<00:00, 1810756.79it/s]


Checking: B


100%|██████████| 60/60 [00:00<00:00, 1525201.45it/s]


Checking: C


...

100%|██████████| 12/12 [00:00<00:00, 474826.87it/s]


Checking: L


100%|██████████| 13/13 [00:00<00:00, 395115.59it/s]


Checking: M


KeyboardInterrupt: 

In [34]:
base_url = "https://dnd-5e.fandom.com"
category_url = f"{base_url}/wiki/Category:Races?from="
headers = {"User-Agent": "Mozilla/5.0"}

output_dir = "../datasets/races"
os.makedirs(output_dir, exist_ok=True)

content(headers, category_url, base_url, output_dir)

100%|██████████| 6/6 [00:01<00:00,  3.24it/s]


Saved: ../datasets/races/A.md


100%|██████████| 1/1 [00:00<00:00,  3.03it/s]


Saved: ../datasets/races/B.md


100%|██████████| 4/4 [00:01<00:00,  3.85it/s]


...

Saved: ../datasets/races/X.md


100%|██████████| 2/2 [00:00<00:00,  2.98it/s]


Saved: ../datasets/races/Y.md


100%|██████████| 88/88 [00:26<00:00,  3.36it/s]

Saved: ../datasets/races/Z.md


In [35]:
base_url = "https://dnd-5e.fandom.com"
category_url = f"{base_url}/wiki/Category:Classes?from="
headers = {"User-Agent": "Mozilla/5.0"}

output_dir = "../datasets/classes"
os.makedirs(output_dir, exist_ok=True)

content(headers, category_url, base_url, output_dir)

100%|██████████| 2/2 [00:00<00:00,  2.52it/s]


Saved: ../datasets/classes/A.md


100%|██████████| 5/5 [00:01<00:00,  3.12it/s]


Saved: ../datasets/classes/B.md


100%|██████████| 2/2 [00:00<00:00,  2.34it/s]


...

Saved: ../datasets/classes/X.md


100%|██████████| 32/32 [00:11<00:00,  2.68it/s]


Saved: ../datasets/classes/Y.md


100%|██████████| 32/32 [00:12<00:00,  2.66it/s]

Saved: ../datasets/classes/Z.md


In [36]:
base_url = "https://dnd-5e.fandom.com"
category_url = f"{base_url}/wiki/Category:Items?from="
headers = {"User-Agent": "Mozilla/5.0"}

output_dir = "../datasets/items"
os.makedirs(output_dir, exist_ok=True)

content(headers, category_url, base_url, output_dir)

100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


Saved: ../datasets/items/A.md


100%|██████████| 7/7 [00:05<00:00,  1.32it/s]


Saved: ../datasets/items/B.md


100%|██████████| 7/7 [00:02<00:00,  2.86it/s]


...

Saved: ../datasets/items/X.md


100%|██████████| 9/9 [00:03<00:00,  2.95it/s]


Saved: ../datasets/items/Y.md


100%|██████████| 9/9 [00:02<00:00,  3.08it/s]

Saved: ../datasets/items/Z.md
